In [ ]:
# Autor M.Sc. Edwin Calle Condori
# %% [markdown]
# # 🤖 AgroIA Bot Completo - Análisis Técnico Profesional con Sentinel-2
#
# **Características:**
# - ✅ Procesamiento robusto de KML/KMZ
# - ✅ Análisis completo con 8 índices espectrales
# - ✅ Informe Word profesional con todas las secciones
# - ✅ Gráficos de alta calidad
# - ✅ Recomendaciones técnicas detalladas

# %%
# INSTALACIÓN DE DEPENDENCIAS
!pip install python-telegram-bot pyngrok earthengine-api geemap geopandas python-docx pillow seaborn nest-asyncio simplekml -q

# %%
# IMPORTACIONES
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import datetime
import json
import tempfile
import os
import logging
from io import BytesIO
import asyncio
import nest_asyncio
import zipfile
import xml.etree.ElementTree as ET
from pyngrok import ngrok
import threading
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from telegram import Update, InputFile, BotCommand
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
import warnings
warnings.filterwarnings('ignore')

# Aplicar nest_asyncio
nest_asyncio.apply()

# Configurar logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# %%
# AUTENTICACIÓN EARTH ENGINE
try:
    ee.Initialize(project='eddycc66') # reemplazar con tu ID de proyecto
    print("✅ Earth Engine inicializado correctamente")
except Exception as e:
    print("❌ Error en inicialización:", str(e))
    print("Intentando autenticación...")
    ee.Authenticate()
    ee.Initialize(project='eddycc66')

# %%
# CONFIGURACIÓN DE TOKENS
TELEGRAM_TOKEN = "# Reemplazar con tu token"  # Reemplazar con tu token
NGROK_TOKEN = "Reemplazar con tu token"  # Reemplazar con tu token
PORT = 8443

# %%
# CLASE ANALIZADOR AGRÍCOLA COMPLETO
class AnalizadorAgricolaCompleto:
    def __init__(self):
        self.output_folder = "/content/agroia_output"
        os.makedirs(self.output_folder, exist_ok=True)
        os.makedirs(f'{self.output_folder}/imagenes', exist_ok=True)

    def crear_geometria_desde_coordenadas(self, lat, lon, buffer_km=5):
        """Crea área circular desde coordenadas GPS"""
        punto = ee.Geometry.Point([lon, lat])
        area = punto.buffer(buffer_km * 1000)
        return area

    def procesar_kml(self, file_bytes, file_name):
        """Procesa archivo KML/KMZ y extrae geometría"""
        try:
            if file_name.lower().endswith('.kmz'):
                with tempfile.NamedTemporaryFile(suffix='.kmz', delete=False) as f:
                    f.write(file_bytes)
                    kmz_path = f.name

                with zipfile.ZipFile(kmz_path, 'r') as kmz:
                    kml_files = [f for f in kmz.namelist() if f.lower().endswith('.kml')]
                    if not kml_files:
                        raise ValueError("No se encontró archivo KML dentro del KMZ")

                    with kmz.open(kml_files[0]) as kml_file:
                        kml_content = kml_file.read()

                    with tempfile.NamedTemporaryFile(mode='wb', suffix='.kml', delete=False) as kml_temp:
                        kml_temp.write(kml_content)
                        kml_path = kml_temp.name

                os.unlink(kmz_path)
            else:
                with tempfile.NamedTemporaryFile(mode='wb', suffix='.kml', delete=False) as f:
                    f.write(file_bytes)
                    kml_path = f.name

            gdf = gpd.read_file(kml_path)

            if gdf.empty:
                raise ValueError("El archivo KML no contiene geometrías válidas")

            geometria = geemap.geopandas_to_ee(gdf)
            os.unlink(kml_path)

            print(f"✅ KML procesado correctamente. Geometrías: {len(gdf)}")
            return geometria.geometry()

        except Exception as e:
            logger.error(f"Error procesando KML: {e}")
            return self._procesar_kml_alternativo(file_bytes)

    def _procesar_kml_alternativo(self, file_bytes):
        """Método alternativo para procesar KML simple"""
        try:
            import re
            kml_content = file_bytes.decode('utf-8')
            coordinates_pattern = r'<coordinates>(.*?)</coordinates>'
            matches = re.findall(coordinates_pattern, kml_content, re.DOTALL)

            if not matches:
                raise ValueError("No se encontraron coordenadas en el KML")

            coords_text = matches[0].strip()
            coords_list = []

            for coord in coords_text.split():
                parts = coord.split(',')
                if len(parts) >= 2:
                    lon, lat = float(parts[0]), float(parts[1])
                    coords_list.append([lon, lat])

            if len(coords_list) < 3:
                raise ValueError("No hay suficientes coordenadas")

            geometria = ee.Geometry.Polygon([coords_list])
            print("✅ KML procesado con método alternativo")
            return geometria

        except Exception as e:
            logger.error(f"Error en método alternativo: {e}")
            return None

    def calcular_indices(self, imagen):
        """Calcula todos los índices espectrales"""
        ndvi = imagen.normalizedDifference(['B8', 'B4']).rename('NDVI')
        gndvi = imagen.normalizedDifference(['B8', 'B3']).rename('GNDVI')
        ndre = imagen.normalizedDifference(['B8A', 'B5']).rename('NDRE')

        evi = imagen.expression(
            '2.5 * (NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1)',
            {'NIR': imagen.select('B8'), 'RED': imagen.select('B4'), 'BLUE': imagen.select('B2')}
        ).rename('EVI')

        savi = imagen.expression(
            '1.5 * (NIR - RED) / (NIR + RED + 0.5)',
            {'NIR': imagen.select('B8'), 'RED': imagen.select('B4')}
        ).rename('SAVI')

        msi = imagen.expression(
            'SWIR1 / NIR',
            {'SWIR1': imagen.select('B11'), 'NIR': imagen.select('B8')}
        ).rename('MSI')

        ndwi = imagen.normalizedDifference(['B3', 'B11']).rename('NDWI')
        nbr = imagen.normalizedDifference(['B8', 'B12']).rename('NBR')

        return imagen.addBands([ndvi, gndvi, ndre, evi, savi, msi, ndwi, nbr])

    def analizar_area(self, geometria, fecha_inicio='2025-01-01', fecha_fin='2025-06-30'):
        """Ejecuta análisis completo del área"""
        try:
            print("📡 Iniciando análisis completo...")

            # 1. SENTINEL-2
            sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR') \
                .filterBounds(geometria) \
                .filterDate(fecha_inicio, fecha_fin) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                .select(['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12'])

            num_imagenes = sentinel2.size().getInfo()
            print(f"📊 Imágenes encontradas: {num_imagenes}")

            if num_imagenes == 0:
                return None, "No hay imágenes disponibles para el área seleccionada"

            imagen_mediana = sentinel2.median().clip(geometria)

            # 2. ÍNDICES
            print("🔬 Calculando índices...")
            imagen_con_indices = self.calcular_indices(imagen_mediana)

            # 3. TOPOGRAFÍA
            print("🗻 Procesando topografía...")
            dem = ee.Image('USGS/SRTMGL1_003').clip(geometria)
            pendiente = ee.Terrain.slope(dem).rename('pendiente')
            aspecto = ee.Terrain.aspect(dem).rename('aspecto')
            dem_completo = dem.addBands([pendiente, aspecto]).rename(['elevacion', 'pendiente', 'aspecto'])

            # 4. ESTADÍSTICAS DE ÍNDICES
            print("📈 Calculando estadísticas...")
            estadisticas_indices = self.calcular_estadisticas_indices(imagen_con_indices, geometria)

            # 5. ESTADÍSTICAS TOPOGRÁFICAS
            estadisticas_topografia = dem_completo.reduceRegion(
                reducer=ee.Reducer.mean().combine(
                    reducer2=ee.Reducer.minMax(),
                    sharedInputs=True
                ).combine(
                    reducer2=ee.Reducer.stdDev(),
                    sharedInputs=True
                ),
                geometry=geometria,
                scale=30,
                maxPixels=1e9
            )
            stats_topo = estadisticas_topografia.getInfo()

            # 6. ZONAS CRÍTICAS
            print("🎯 Identificando zonas críticas...")
            zonas_intervencion = self.identificar_zonas_intervencion(imagen_con_indices, dem_completo)

            # 7. CALCULAR ÁREAS
            area_total = geometria.area().divide(10000).getInfo()

            areas = {
                'alta': self.calcular_area_zona(zonas_intervencion.select('prioridad_alta'), geometria, 'prioridad_alta'),
                'media': self.calcular_area_zona(zonas_intervencion.select('prioridad_media'), geometria, 'prioridad_media'),
                'baja': self.calcular_area_zona(zonas_intervencion.select('prioridad_baja'), geometria, 'prioridad_baja'),
                'erosion': self.calcular_area_zona(zonas_intervencion.select('riesgo_erosion'), geometria, 'riesgo_erosion')
            }

            # 8. CREAR DATAFRAMES
            df_estadisticas = pd.DataFrame()
            for indice, stats in estadisticas_indices.items():
                df_estadisticas.loc[indice, 'Media'] = stats.get(f'{indice}_mean', 0)
                df_estadisticas.loc[indice, 'Mínimo'] = stats.get(f'{indice}_min', 0)
                df_estadisticas.loc[indice, 'Máximo'] = stats.get(f'{indice}_max', 0)
                df_estadisticas.loc[indice, 'Desviación'] = stats.get(f'{indice}_stdDev', 0)

            df_intervenciones = pd.DataFrame({
                'Prioridad': ['ALTA', 'MEDIA', 'BAJA', 'Riesgo Erosión'],
                'Área (ha)': [areas['alta'], areas['media'], areas['baja'], areas['erosion']],
                'Acción Recomendada': [
                    'Riego inmediato + Fertilización + Cobertura',
                    'Riego focalizado + Monitoreo',
                    'Optimizar fertilización',
                    'Prácticas conservacionistas urgentes'
                ],
                'Plazo': ['INMEDIATO', '1-2 semanas', '1 mes', 'INMEDIATO']
            })
            df_intervenciones['% del Total'] = (df_intervenciones['Área (ha)'] / area_total * 100).round(2)

            # 9. GENERAR GRÁFICOS
            print("📊 Generando gráficos...")
            self.crear_graficos_completos(df_estadisticas, df_intervenciones, stats_topo, area_total)

            # 10. GENERAR INFORME COMPLETO
            print("📝 Generando informe Word completo...")
            informe_path = self.generar_informe_word_completo(
                df_estadisticas, df_intervenciones, stats_topo, area_total
            )

            print("✅ Análisis completado exitosamente")
            return informe_path, None

        except Exception as e:
            logger.error(f"Error en análisis: {e}")
            return None, f"Error: {str(e)}"

    def calcular_estadisticas_indices(self, imagen, geometry):
        """Calcula estadísticas para todos los índices"""
        indices = ['NDVI', 'GNDVI', 'NDRE', 'EVI', 'SAVI', 'MSI', 'NDWI', 'NBR']
        estadisticas = {}

        for indice in indices:
            stats = imagen.select(indice).reduceRegion(
                reducer=ee.Reducer.mean().combine(
                    reducer2=ee.Reducer.minMax(),
                    sharedInputs=True
                ).combine(
                    reducer2=ee.Reducer.stdDev(),
                    sharedInputs=True
                ),
                geometry=geometry,
                scale=30,
                maxPixels=1e9
            )
            estadisticas[indice] = stats.getInfo()

        return estadisticas

    def identificar_zonas_intervencion(self, imagen_indices, dem_datos):
        """Identifica zonas que requieren intervención"""
        prioridad_alta = imagen_indices.select('NDVI').lt(0.3).And(
            imagen_indices.select('NDWI').lt(-0.1)
        ).And(
            imagen_indices.select('MSI').gt(1.2)
        ).rename('prioridad_alta')

        prioridad_media = imagen_indices.select('NDVI').gte(0.3).And(
            imagen_indices.select('NDVI').lt(0.5)
        ).And(
            imagen_indices.select('NDWI').lt(0).Or(
                imagen_indices.select('NDRE').lt(0.25)
            )
        ).rename('prioridad_media')

        prioridad_baja = imagen_indices.select('NDVI').gte(0.5).And(
            imagen_indices.select('NDVI').lt(0.6)
        ).And(
            imagen_indices.select('NDRE').lt(0.3).Or(
                imagen_indices.select('MSI').gt(1.0)
            )
        ).rename('prioridad_baja')

        riesgo_erosion = dem_datos.select('pendiente').gt(15).And(
            imagen_indices.select('NDVI').lt(0.4)
        ).rename('riesgo_erosion')

        return prioridad_alta.addBands([prioridad_media, prioridad_baja, riesgo_erosion])

    def calcular_area_zona(self, mascara, geometry, nombre):
        """Calcula área en hectáreas"""
        try:
            area_pixeles = mascara.multiply(ee.Image.pixelArea()).reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=geometry,
                scale=30,
                maxPixels=1e9
            )
            area_ha = ee.Number(area_pixeles.get(nombre)).divide(10000)
            return area_ha.getInfo() or 0
        except:
            return 0

    def crear_graficos_completos(self, df_est, df_int, stats_topo, area_total):
        """Crea todos los gráficos para el informe"""
        try:
            # Gráfico 1: Comparación de índices
            fig1, ax1 = plt.subplots(figsize=(12, 6))
            colores = ['#2ecc71', '#27ae60', '#3498db', '#1abc9c', '#16a085', '#e74c3c', '#3498db', '#f39c12']
            df_est['Media'].plot(kind='bar', color=colores, edgecolor='black', ax=ax1)
            ax1.set_title('Valores Promedio de Índices Espectrales', fontsize=16, fontweight='bold', pad=20)
            ax1.set_ylabel('Valor del Índice', fontsize=12, fontweight='bold')
            ax1.set_xlabel('Índice Espectral', fontsize=12, fontweight='bold')
            ax1.grid(axis='y', alpha=0.3, linestyle='--')
            ax1.axhline(y=0, color='black', linewidth=0.8)
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.savefig(f'{self.output_folder}/imagenes/01_comparacion_indices.png', dpi=300, bbox_inches='tight')
            plt.close()

            # Gráfico 2: Plan de intervención
            fig2, (ax2a, ax2b) = plt.subplots(1, 2, figsize=(16, 6))
            colores_prioridad = ['#FF0000', '#FFA500', '#FFFF00', '#8B4513']
            ax2a.barh(df_int['Prioridad'], df_int['Área (ha)'], color=colores_prioridad, edgecolor='black', linewidth=1.5)
            ax2a.set_xlabel('Área (hectáreas)', fontsize=12, fontweight='bold')
            ax2a.set_title('Extensión de Zonas por Prioridad', fontsize=14, fontweight='bold')
            ax2a.grid(axis='x', alpha=0.3)
            for i, v in enumerate(df_int['Área (ha)']):
                ax2a.text(v + 5, i, f'{v:.1f} ha', va='center', fontweight='bold')

            ax2b.pie(df_int['Área (ha)'], labels=df_int['Prioridad'], autopct='%1.1f%%',
                    colors=colores_prioridad, startangle=90, textprops={'fontsize': 11, 'fontweight': 'bold'})
            ax2b.set_title('Distribución Porcentual', fontsize=14, fontweight='bold')
            plt.tight_layout()
            plt.savefig(f'{self.output_folder}/imagenes/02_plan_intervencion.png', dpi=300, bbox_inches='tight')
            plt.close()

            # Gráfico 3: Salud de cultivos
            fig3, axes = plt.subplots(2, 2, figsize=(14, 10))
            indices_clave = ['NDVI', 'NDWI', 'NDRE', 'MSI']
            titulos = ['Vigor Vegetal (NDVI)', 'Contenido de Humedad (NDWI)',
                      'Estado Nutricional (NDRE)', 'Estrés Hídrico (MSI)']
            colores_ind = ['green', 'blue', 'orange', 'red']

            for idx, (indice, titulo, color) in enumerate(zip(indices_clave, titulos, colores_ind)):
                ax = axes[idx // 2, idx % 2]
                valores = [df_est.loc[indice, 'Mínimo'], df_est.loc[indice, 'Media'], df_est.loc[indice, 'Máximo']]
                bars = ax.bar(['Mínimo', 'Promedio', 'Máximo'], valores, color=[color]*3,
                             alpha=0.8, edgecolor='black', linewidth=1.5)
                ax.set_title(titulo, fontsize=12, fontweight='bold')
                ax.set_ylabel('Valor', fontsize=10, fontweight='bold')
                ax.grid(axis='y', alpha=0.3)
                ax.axhline(y=0, color='black', linewidth=0.8)
                for bar, val in zip(bars, valores):
                    height = bar.get_height()
                    ax.text(bar.get_x() + bar.get_width()/2., height, f'{val:.3f}',
                           ha='center', va='bottom', fontweight='bold')

            plt.tight_layout()
            plt.savefig(f'{self.output_folder}/imagenes/03_analisis_salud_cultivos.png', dpi=300, bbox_inches='tight')
            plt.close()

            # Gráfico 4: Topografía
            fig4, (ax4a, ax4b) = plt.subplots(1, 2, figsize=(14, 6))
            elevaciones = [stats_topo['elevacion_min'], stats_topo['elevacion_mean'], stats_topo['elevacion_max']]
            ax4a.bar(['Mínima', 'Media', 'Máxima'], elevaciones,
                    color=['#27ae60', '#f39c12', '#c0392b'], edgecolor='black', linewidth=1.5)
            ax4a.set_ylabel('Elevación (m.s.n.m.)', fontsize=12, fontweight='bold')
            ax4a.set_title('Perfil Altitudinal', fontsize=14, fontweight='bold')
            ax4a.grid(axis='y', alpha=0.3)
            for i, v in enumerate(elevaciones):
                ax4a.text(i, v + 5, f'{v:.0f} m', ha='center', fontweight='bold')

            pendientes = [stats_topo['pendiente_mean'], stats_topo['pendiente_max']]
            bars = ax4b.bar(['Pendiente\nMedia', 'Pendiente\nMáxima'], pendientes,
                           color=['#3498db', '#e74c3c'], edgecolor='black', linewidth=1.5)
            ax4b.set_ylabel('Pendiente (%)', fontsize=12, fontweight='bold')
            ax4b.set_title('Análisis de Pendientes', fontsize=14, fontweight='bold')
            ax4b.axhline(y=15, color='orange', linestyle='--', linewidth=2, label='Umbral Riesgo (15%)')
            ax4b.grid(axis='y', alpha=0.3)
            ax4b.legend()
            for bar, val in zip(bars, pendientes):
                height = bar.get_height()
                ax4b.text(bar.get_x() + bar.get_width()/2., height, f'{val:.1f}%',
                         ha='center', va='bottom', fontweight='bold')

            plt.tight_layout()
            plt.savefig(f'{self.output_folder}/imagenes/04_topografia_riesgos.png', dpi=300, bbox_inches='tight')
            plt.close()

            print("✅ Gráficos generados correctamente")

        except Exception as e:
            print(f"⚠️ Error creando gráficos: {e}")

    def generar_informe_word_completo(self, df_est, df_int, stats_topo, area_total):
        """Genera informe Word completo profesional"""
        try:
            doc = Document()
            style = doc.styles['Normal']
            style.font.name = 'Arial'
            style.font.size = Pt(11)

            # === PORTADA ===
            portada = doc.add_heading('INFORME TÉCNICO DE ANÁLISIS AGRÍCOLA', 0)
            portada.alignment = WD_ALIGN_PARAGRAPH.CENTER

            doc.add_paragraph()
            titulo_2 = doc.add_heading('Análisis Multitemporal con Sentinel-2 y DEM', level=1)
            titulo_2.alignment = WD_ALIGN_PARAGRAPH.CENTER

            doc.add_paragraph()
            info = doc.add_paragraph()
            info.add_run('Proyecto: ').bold = True
            info.add_run('AgroIA - Análisis Satelital Profesional\n')
            info.add_run('Período de análisis: ').bold = True
            info.add_run('Enero - Junio 2025\n')
            info.add_run('Sensor: ').bold = True
            info.add_run('Sentinel-2 L2A (10-30m) + SRTM DEM (30m)\n')
            info.add_run('Fecha de elaboración: ').bold = True
            info.add_run(datetime.datetime.now().strftime('%d/%m/%Y %H:%M'))
            info.alignment = WD_ALIGN_PARAGRAPH.CENTER

            doc.add_page_break()

            # === ÍNDICE ===
            doc.add_heading('ÍNDICE', level=1)
            indice_items = [
                '1. RESUMEN EJECUTIVO',
                '2. DESCRIPCIÓN DEL ÁREA DE ESTUDIO',
                '3. METODOLOGÍA Y FUENTES DE DATOS',
                '4. ANÁLISIS DE ÍNDICES ESPECTRALES',
                '5. PLAN DE INTERVENCIÓN FOCALIZADO',
                '6. RECOMENDACIONES TÉCNICAS',
                '7. CONCLUSIONES Y PRÓXIMOS PASOS'
            ]
            for item in indice_items:
                doc.add_paragraph(item, style='List Number')

            doc.add_page_break()

            # === 1. RESUMEN EJECUTIVO ===
            doc.add_heading('1. RESUMEN EJECUTIVO', level=1)

            p = doc.add_paragraph()
            p.add_run('Este informe presenta los resultados del análisis satelital multitemporal realizado con tecnología Sentinel-2, ')
            p.add_run('integrando 8 índices espectrales y análisis topográfico detallado para identificar zonas que requieren ')
            p.add_run('intervención inmediata en el área de estudio.')

            doc.add_paragraph()
            doc.add_heading('Hallazgos Principales:', level=2)

            hallazgos = doc.add_paragraph()
            hallazgos.add_run(f'• Área total analizada: ').bold = True
            hallazgos.add_run(f'{area_total:,.1f} hectáreas\n')

            ndvi_val = df_est.loc['NDVI', 'Media']
            estado_ndvi = 'EXCELENTE' if ndvi_val > 0.6 else ('MODERADO' if ndvi_val > 0.3 else 'CRÍTICO')
            hallazgos.add_run(f'• NDVI promedio: ').bold = True
            hallazgos.add_run(f'{ndvi_val:.3f} - Estado: ')
            run_estado = hallazgos.add_run(f'{estado_ndvi}\n')
            if estado_ndvi == 'CRÍTICO':
                run_estado.font.color.rgb = RGBColor(255, 0, 0)
            elif estado_ndvi == 'MODERADO':
                run_estado.font.color.rgb = RGBColor(255, 140, 0)
            else:
                run_estado.font.color.rgb = RGBColor(0, 128, 0)

            hallazgos.add_run(f'• Zonas de prioridad alta: ').bold = True
            hallazgos.add_run(f'{df_int.loc[0, "Área (ha)"]:.1f} ha ({df_int.loc[0, "% del Total"]:.1f}%)\n')

            hallazgos.add_run(f'• Pendiente media: ').bold = True
            hallazgos.add_run(f'{stats_topo["pendiente_mean"]:.1f}%\n')

            hallazgos.add_run(f'• Estado de humedad: ').bold = True
            ndwi_val = df_est.loc['NDWI', 'Media']
            if ndwi_val > 0.3:
                hallazgos.add_run('Saturación hídrica\n')
            elif ndwi_val > 0:
                hallazgos.add_run('Óptimo\n')
            else:
                hallazgos.add_run('Déficit hídrico detectado\n')

            doc.add_page_break()

            # === 2. DESCRIPCIÓN DEL ÁREA ===
            doc.add_heading('2. DESCRIPCIÓN DEL ÁREA DE ESTUDIO', level=1)

            p = doc.add_paragraph()
            p.add_run(f'El área de estudio comprende {area_total:,.1f} hectáreas. ')
            p.add_run('Este análisis satelital permite identificar patrones espaciales de vegetación y diseñar ')
            p.add_run('estrategias de manejo diferenciado basadas en datos objetivos.')

            doc.add_heading('2.1 Características Topográficas', level=2)

            tabla_topo = doc.add_table(rows=6, cols=2)
            tabla_topo.style = 'Light Grid Accent 1'

            datos_topo = [
                ['Extensión Total', f'{area_total:,.1f} hectáreas'],
                ['Elevación Mínima', f'{stats_topo["elevacion_min"]:.0f} m.s.n.m.'],
                ['Elevación Máxima', f'{stats_topo["elevacion_max"]:.0f} m.s.n.m.'],
                ['Elevación Media', f'{stats_topo["elevacion_mean"]:.0f} m'],
                ['Pendiente Media', f'{stats_topo["pendiente_mean"]:.1f}%'],
                ['Pendiente Máxima', f'{stats_topo["pendiente_max"]:.1f}%']
            ]

            for i, (campo, valor) in enumerate(datos_topo):
                tabla_topo.rows[i].cells[0].text = campo
                tabla_topo.rows[i].cells[0].paragraphs[0].runs[0].font.bold = True
                tabla_topo.rows[i].cells[1].text = valor

            doc.add_paragraph()

            # Insertar gráfico topográfico
            try:
                doc.add_picture(f'{self.output_folder}/imagenes/04_topografia_riesgos.png', width=Inches(6))
                last_paragraph = doc.paragraphs[-1]
                last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
                caption = doc.add_paragraph('Figura 1: Análisis topográfico - Elevación y pendientes del área')
                caption.alignment = WD_ALIGN_PARAGRAPH.CENTER
            except Exception as e:
                doc.add_paragraph(f'⚠️ Gráfico no disponible')

            doc.add_page_break()

            # === 3. METODOLOGÍA ===
            doc.add_heading('3. METODOLOGÍA Y FUENTES DE DATOS', level=1)

            p = doc.add_paragraph()
            p.add_run('El análisis se realizó utilizando la plataforma Google Earth Engine, procesando imágenes ')
            p.add_run('satelitales Sentinel-2 Nivel 2A (corrección atmosférica) del período enero-junio 2025. ')
            p.add_run('Se aplicaron filtros de nubosidad (<20%) para garantizar la calidad de los datos.')

            doc.add_heading('3.1 Índices Espectrales Calculados', level=2)

            tabla_indices = doc.add_table(rows=9, cols=3)
            tabla_indices.style = 'Light Grid Accent 1'

            tabla_indices.rows[0].cells[0].text = 'Índice'
            tabla_indices.rows[0].cells[1].text = 'Significado'
            tabla_indices.rows[0].cells[2].text = 'Rango Normal'

            for cell in tabla_indices.rows[0].cells:
                cell.paragraphs[0].runs[0].font.bold = True

            indices_info = [
                ['NDVI', 'Vigor y biomasa vegetal', '0.3 - 0.8'],
                ['GNDVI', 'Contenido de clorofila', '0.3 - 0.7'],
                ['NDRE', 'Estado nutricional (N)', '0.2 - 0.5'],
                ['EVI', 'Vegetación densa', '0.2 - 0.8'],
                ['SAVI', 'Corrección por suelo', '0.2 - 0.6'],
                ['MSI', 'Estrés hídrico', '0.4 - 2.0'],
                ['NDWI', 'Contenido de humedad', '-0.3 - 0.3'],
                ['NBR', 'Áreas degradadas', '-1.0 - 1.0']
            ]

            for i, (indice, signif, rango) in enumerate(indices_info, 1):
                tabla_indices.rows[i].cells[0].text = indice
                tabla_indices.rows[i].cells[1].text = signif
                tabla_indices.rows[i].cells[2].text = rango

            doc.add_heading('3.2 Procesamiento de Datos', level=2)

            p = doc.add_paragraph()
            p.add_run('• Filtrado de nubosidad: <20%\n')
            p.add_run('• Composición temporal: Mediana del período\n')
            p.add_run('• Resolución espacial: 10-30 metros\n')
            p.add_run('• Modelo digital de elevación: SRTM 30m\n')

            doc.add_page_break()

            # === 4. ANÁLISIS DE ÍNDICES ===
            doc.add_heading('4. ANÁLISIS DE ÍNDICES ESPECTRALES', level=1)

            # Insertar gráfico de comparación
            try:
                doc.add_picture(f'{self.output_folder}/imagenes/01_comparacion_indices.png', width=Inches(6))
                last_paragraph = doc.paragraphs[-1]
                last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
                caption = doc.add_paragraph('Figura 2: Valores promedio de índices espectrales')
                caption.alignment = WD_ALIGN_PARAGRAPH.CENTER
            except:
                doc.add_paragraph(f'⚠️ Gráfico no disponible')

            doc.add_paragraph()

            # Tabla de estadísticas
            doc.add_heading('4.1 Estadísticas Detalladas', level=2)

            tabla_stats = doc.add_table(rows=len(df_est)+1, cols=5)
            tabla_stats.style = 'Light Grid Accent 1'

            encabezados = ['Índice', 'Media', 'Mínimo', 'Máximo', 'Desv. Est.']
            for i, enc in enumerate(encabezados):
                tabla_stats.rows[0].cells[i].text = enc
                tabla_stats.rows[0].cells[i].paragraphs[0].runs[0].font.bold = True

            for i, indice in enumerate(df_est.index, 1):
                tabla_stats.rows[i].cells[0].text = indice
                tabla_stats.rows[i].cells[1].text = f'{df_est.loc[indice, "Media"]:.3f}'
                tabla_stats.rows[i].cells[2].text = f'{df_est.loc[indice, "Mínimo"]:.3f}'
                tabla_stats.rows[i].cells[3].text = f'{df_est.loc[indice, "Máximo"]:.3f}'
                tabla_stats.rows[i].cells[4].text = f'{df_est.loc[indice, "Desviación"]:.3f}'

            doc.add_paragraph()

            # Insertar análisis de salud
            try:
                doc.add_picture(f'{self.output_folder}/imagenes/03_analisis_salud_cultivos.png', width=Inches(6))
                last_paragraph = doc.paragraphs[-1]
                last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
                caption = doc.add_paragraph('Figura 3: Análisis comparativo de indicadores clave de salud vegetal')
                caption.alignment = WD_ALIGN_PARAGRAPH.CENTER
            except:
                doc.add_paragraph(f'⚠️ Gráfico no disponible')

            doc.add_paragraph()

            # Interpretación automática
            doc.add_heading('4.2 Interpretación de Resultados', level=2)

            # NDVI
            ndvi_mean = df_est.loc['NDVI', 'Media']
            p = doc.add_paragraph()
            p.add_run('NDVI (Vigor Vegetal): ').bold = True
            p.add_run(f'{ndvi_mean:.3f} - ')

            if ndvi_mean > 0.6:
                run = p.add_run('EXCELENTE\n')
                run.font.color.rgb = RGBColor(0, 128, 0)
                run.bold = True
                p.add_run('✓ Vegetación vigorosa con alta actividad fotosintética\n')
                p.add_run('✓ Biomasa vegetal óptima\n')
                p.add_run('✓ Recomendación: Mantener prácticas actuales de manejo\n')
            elif ndvi_mean > 0.3:
                run = p.add_run('MODERADO\n')
                run.font.color.rgb = RGBColor(255, 140, 0)
                run.bold = True
                p.add_run('⚠ Estrés vegetal moderado detectado\n')
                p.add_run('⚠ Biomasa por debajo del óptimo\n')
                p.add_run('⚠ Recomendación: Revisar sistemas de riego y plan de fertilización\n')
            else:
                run = p.add_run('CRÍTICO\n')
                run.font.color.rgb = RGBColor(255, 0, 0)
                run.bold = True
                p.add_run('✗ Vegetación severamente degradada o suelo desnudo\n')
                p.add_run('✗ Actividad fotosintética mínima\n')
                p.add_run('✗ Recomendación: Intervención urgente requerida - Riego y fertilización inmediata\n')

            doc.add_paragraph()

            # NDWI
            ndwi_mean = df_est.loc['NDWI', 'Media']
            p = doc.add_paragraph()
            p.add_run('NDWI (Contenido de Humedad): ').bold = True
            p.add_run(f'{ndwi_mean:.3f} - ')

            if ndwi_mean > 0.3:
                run = p.add_run('ALTA HUMEDAD\n')
                run.font.color.rgb = RGBColor(0, 0, 255)
                run.bold = True
                p.add_run('⚠ Posible saturación hídrica\n')
                p.add_run('⚠ Verificar sistemas de drenaje\n')
                p.add_run('⚠ Riesgo de enfermedades por exceso de humedad\n')
            elif ndwi_mean > 0:
                run = p.add_run('ÓPTIMO\n')
                run.font.color.rgb = RGBColor(0, 128, 0)
                run.bold = True
                p.add_run('✓ Contenido de humedad adecuado para cultivos\n')
                p.add_run('✓ Balance hídrico apropiado\n')
            else:
                run = p.add_run('DÉFICIT HÍDRICO\n')
                run.font.color.rgb = RGBColor(255, 0, 0)
                run.bold = True
                p.add_run('✗ Estrés hídrico severo detectado\n')
                p.add_run('✗ Contenido de agua insuficiente en tejidos vegetales\n')
                p.add_run('✗ Implementar sistema de riego adicional urgente\n')

            doc.add_paragraph()

            # NDRE
            ndre_mean = df_est.loc['NDRE', 'Media']
            p = doc.add_paragraph()
            p.add_run('NDRE (Estado Nutricional - Nitrógeno): ').bold = True
            p.add_run(f'{ndre_mean:.3f} - ')

            if ndre_mean > 0.3:
                run = p.add_run('ÓPTIMO\n')
                run.font.color.rgb = RGBColor(0, 128, 0)
                run.bold = True
                p.add_run('✓ Contenido de nitrógeno adecuado en follaje\n')
                p.add_run('✓ Coloración verde intensa indicando buena nutrición\n')
            elif ndre_mean > 0.2:
                run = p.add_run('MODERADO\n')
                run.font.color.rgb = RGBColor(255, 140, 0)
                run.bold = True
                p.add_run('⚠ Deficiencia nutricional leve\n')
                p.add_run('⚠ Considerar aplicación de fertilización nitrogenada\n')
                p.add_run('⚠ Dosis recomendada: 60-80 kg N/ha\n')
            else:
                run = p.add_run('DEFICIENTE\n')
                run.font.color.rgb = RGBColor(255, 0, 0)
                run.bold = True
                p.add_run('✗ Deficiencia severa de nitrógeno\n')
                p.add_run('✗ Clorosis foliar evidente\n')
                p.add_run('✗ Aplicar fertilizantes nitrogenados urgentemente\n')
                p.add_run('✗ Dosis recomendada: 100-120 kg N/ha en aplicaciones fraccionadas\n')

            doc.add_paragraph()

            # MSI
            msi_mean = df_est.loc['MSI', 'Media']
            p = doc.add_paragraph()
            p.add_run('MSI (Índice de Estrés Hídrico): ').bold = True
            p.add_run(f'{msi_mean:.3f} - ')

            if msi_mean < 1.0:
                run = p.add_run('ÓPTIMO\n')
                run.font.color.rgb = RGBColor(0, 128, 0)
                run.bold = True
                p.add_run('✓ Sin estrés hídrico significativo\n')
            elif msi_mean < 1.5:
                run = p.add_run('ESTRÉS MODERADO\n')
                run.font.color.rgb = RGBColor(255, 140, 0)
                run.bold = True
                p.add_run('⚠ Inicio de estrés hídrico\n')
                p.add_run('⚠ Monitorear evolución y ajustar riego\n')
            else:
                run = p.add_run('ESTRÉS SEVERO\n')
                run.font.color.rgb = RGBColor(255, 0, 0)
                run.bold = True
                p.add_run('✗ Estrés hídrico crítico\n')
                p.add_run('✗ Riego urgente necesario\n')

            doc.add_page_break()

            # === 5. PLAN DE INTERVENCIÓN ===
            doc.add_heading('5. PLAN DE INTERVENCIÓN - MAPA FOCALIZADO', level=1)

            p = doc.add_paragraph()
            p.add_run('IMPORTANTE: ').bold = True
            p.add_run('Este mapa identifica las zonas exactas que requieren intervención inmediata. ')
            p.add_run('Las áreas han sido clasificadas según el nivel de estrés vegetal detectado mediante ')
            p.add_run('análisis multitemporal combinando múltiples índices espectrales.')

            doc.add_paragraph()

            # Insertar gráfico de plan
            try:
                doc.add_picture(f'{self.output_folder}/imagenes/02_plan_intervencion.png', width=Inches(6))
                last_paragraph = doc.paragraphs[-1]
                last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
                caption = doc.add_paragraph('Figura 4: Distribución de zonas según prioridad de intervención')
                caption.alignment = WD_ALIGN_PARAGRAPH.CENTER
            except:
                doc.add_paragraph(f'⚠️ Gráfico no disponible')

            doc.add_paragraph()

            # Tabla de intervenciones
            doc.add_heading('5.1 Detalle de Zonas Identificadas', level=2)

            tabla_interv = doc.add_table(rows=len(df_int)+1, cols=5)
            tabla_interv.style = 'Light Grid Accent 1'

            tabla_interv.rows[0].cells[0].text = 'Prioridad'
            tabla_interv.rows[0].cells[1].text = 'Área (ha)'
            tabla_interv.rows[0].cells[2].text = '% Total'
            tabla_interv.rows[0].cells[3].text = 'Acción Recomendada'
            tabla_interv.rows[0].cells[4].text = 'Plazo'

            for cell in tabla_interv.rows[0].cells:
                cell.paragraphs[0].runs[0].font.bold = True

            for i, row in df_int.iterrows():
                tabla_interv.rows[i+1].cells[0].text = row['Prioridad']
                tabla_interv.rows[i+1].cells[1].text = f"{row['Área (ha)']:.1f}"
                tabla_interv.rows[i+1].cells[2].text = f"{row['% del Total']:.1f}%"
                tabla_interv.rows[i+1].cells[3].text = row['Acción Recomendada']
                tabla_interv.rows[i+1].cells[4].text = row['Plazo']

            doc.add_paragraph()

            # Criterios de clasificación
            doc.add_heading('5.2 Criterios de Clasificación', level=2)

            p = doc.add_paragraph()
            p.add_run('🔴 Prioridad ALTA: ').bold = True
            p.add_run('NDVI < 0.3 + NDWI < -0.1 + MSI > 1.2\n')
            p.add_run('Múltiples factores de estrés simultáneos: baja biomasa vegetal + déficit hídrico severo + ')
            p.add_run('estrés hídrico crítico. Requiere acción inmediata para evitar pérdidas.\n\n')

            p.add_run('🟠 Prioridad MEDIA: ').bold = True
            p.add_run('0.3 ≤ NDVI < 0.5 + (NDWI < 0 O NDRE < 0.25)\n')
            p.add_run('Estrés moderado que requiere ajustes en el manejo dentro de 1-2 semanas. ')
            p.add_run('Situación controlable con intervención oportuna.\n\n')

            p.add_run('🟡 Prioridad BAJA: ').bold = True
            p.add_run('0.5 ≤ NDVI < 0.6 + deficiencias leves\n')
            p.add_run('Vegetación en condiciones aceptables pero con margen de optimización. ')
            p.add_run('Ajustes preventivos mediante calibración de fertilización.\n\n')

            p.add_run('🟤 Riesgo de Erosión: ').bold = True
            p.add_run('Pendiente > 15% + NDVI < 0.4\n')
            p.add_run('Alto riesgo de pérdida de suelo por erosión. Implementar urgentemente prácticas ')
            p.add_run('conservacionistas: terrazas, barreras vivas, cobertura vegetal permanente.\n')

            doc.add_page_break()

            # === 6. RECOMENDACIONES TÉCNICAS ===
            doc.add_heading('6. RECOMENDACIONES TÉCNICAS', level=1)

            doc.add_heading('6.1 Acciones Inmediatas (0-2 semanas)', level=2)

            recom = doc.add_paragraph()

            if df_int.loc[0, 'Área (ha)'] > 0:
                recom.add_run('1. RIEGO FOCALIZADO - PRIORIDAD CRÍTICA\n').bold = True
                recom.add_run(f'   • Área afectada: {df_int.loc[0, "Área (ha)"]:.1f} hectáreas ')
                recom.add_run(f'({df_int.loc[0, "% del Total"]:.1f}% del área total)\n')
                recom.add_run('   • Implementar riego inmediato en zonas rojas del mapa\n')
                recom.add_run('   • Frecuencia: Riego diario hasta recuperación de NDVI > 0.3\n')
                recom.add_run('   • Lámina recomendada: 25-30 mm según tipo de suelo\n')
                recom.add_run('   • Monitoreo diario de humedad del suelo (tensiómetros)\n')
                recom.add_run('   • Priorizar horas de menor evapotranspiración (temprano/tarde)\n\n')

            if ndre_mean < 0.25:
                recom.add_run('2. FERTILIZACIÓN NITROGENADA URGENTE\n').bold = True
                recom.add_run('   • Aplicar en zonas con NDRE < 0.25\n')
                recom.add_run('   • Dosis base: 100-120 kg N/ha\n')
                recom.add_run('   • Modalidad: Aplicación fraccionada en 2-3 dosis\n')
                recom.add_run('   • Fuente recomendada: Urea (46% N) o nitrato de amonio\n')
                recom.add_run('   • Timing: Primera aplicación inmediata, seguimiento cada 15 días\n')
                recom.add_run('   • Aplicar con humedad adecuada para evitar pérdidas por volatilización\n\n')

            if df_int.loc[3, 'Área (ha)'] > 0:
                recom.add_run('3. CONTROL DE EROSIÓN - ACCIÓN PREVENTIVA\n').bold = True
                recom.add_run(f'   • Área en riesgo: {df_int.loc[3, "Área (ha)"]:.1f} hectáreas\n')
                recom.add_run('   • Implementar curvas de nivel o terrazas en pendientes >15%\n')
                recom.add_run('   • Establecer barreras vivas con especies nativas\n')
                recom.add_run('   • Cobertura vegetal permanente (cultivos de cobertura)\n')
                recom.add_run('   • Evitar labranza convencional en estas zonas\n\n')

            if df_int.loc[1, 'Área (ha)'] > 0:
                recom.add_run('4. MANEJO DE ZONAS PRIORIDAD MEDIA\n').bold = True
                recom.add_run(f'   • Área: {df_int.loc[1, "Área (ha)"]:.1f} hectáreas\n')
                recom.add_run('   • Riego suplementario programado\n')
                recom.add_run('   • Ajuste de fertilización según análisis foliares\n')
                recom.add_run('   • Monitoreo semanal de evolución\n\n')

            doc.add_heading('6.2 Monitoreo Continuo y Seguimiento', level=2)

            p = doc.add_paragraph()
            p.add_run('Se recomienda implementar un programa de monitoreo satelital mensual para:\n\n')

            p.add_run('• Evaluación de efectividad: ').bold = True
            p.add_run('Verificar respuesta de las zonas intervenidas mediante comparación de índices NDVI/NDWI\n')

            p.add_run('• Detección temprana: ').bold = True
            p.add_run('Identificar nuevas áreas de estrés antes de que se vuelvan críticas\n')

            p.add_run('• Ajuste dinámico: ').bold = True
            p.add_run('Modificar estrategias de manejo según respuesta observada\n')

            p.add_run('• Registro histórico: ').bold = True
            p.add_run('Generar base de datos temporal para análisis de tendencias\n')

            p.add_run('• Validación en campo: ').bold = True
            p.add_run('Correlacionar datos satelitales con mediciones in situ\n')

            doc.add_heading('6.3 Recomendaciones por Tipo de Cultivo', level=2)

            p = doc.add_paragraph()
            p.add_run('Las recomendaciones específicas deben ajustarse según:\n')
            p.add_run('• Tipo de cultivo y estado fenológico actual\n')
            p.add_run('• Características del suelo (textura, pH, materia orgánica)\n')
            p.add_run('• Disponibilidad de recursos hídricos\n')
            p.add_run('• Condiciones climáticas previstas\n')
            p.add_run('• Objetivos productivos y económicos\n')

            doc.add_page_break()

            # === 7. CONCLUSIONES ===
            doc.add_heading('7. CONCLUSIONES Y PRÓXIMOS PASOS', level=1)

            concl = doc.add_paragraph()
            concl.add_run('DIAGNÓSTICO GENERAL:\n').bold = True
            concl.add_run(f'El análisis satelital multitemporal ha identificado con precisión las zonas que requieren ')
            concl.add_run(f'intervención en el área de estudio de {area_total:,.1f} hectáreas. ')

            area_critica = df_int.loc[0, 'Área (ha)']
            pct_critica = df_int.loc[0, '% del Total']

            if area_critica > 0:
                concl.add_run(f'Se detectaron {area_critica:.1f} hectáreas ({pct_critica:.1f}% del área total) ')
                concl.add_run('con prioridad ALTA que necesitan acción inmediata para evitar pérdidas productivas.\n\n')

            concl.add_run('ESTADO GENERAL DEL ÁREA:\n').bold = True
            if ndvi_mean > 0.6:
                concl.add_run('EXCELENTE - ')
                concl.add_run('La vegetación presenta vigor óptimo en la mayoría del territorio analizado. ')
                concl.add_run('Se recomienda mantener las prácticas actuales de manejo y realizar ajustes ')
                concl.add_run('focalizados en las zonas de menor rendimiento.\n\n')
            elif ndvi_mean > 0.3:
                concl.add_run('MODERADO - ')
                concl.add_run('El área presenta condiciones aceptables pero con presencia significativa de estrés vegetal. ')
                concl.add_run('Es fundamental implementar las recomendaciones propuestas en el plazo indicado ')
                concl.add_run('para evitar deterioro adicional.\n\n')
            else:
                concl.add_run('CRÍTICO - ')
                run_critico = concl.add_run('El área requiere intervención urgente e integral. ')
                run_critico.font.color.rgb = RGBColor(255, 0, 0)
                run_critico.bold = True
                concl.add_run('La combinación de bajo NDVI, déficit hídrico y posibles deficiencias nutricionales ')
                concl.add_run('indica un estado vegetativo severamente comprometido. La acción inmediata es ')
                concl.add_run('imprescindible para la recuperación del área.\n\n')

            concl.add_run('FACTORES LIMITANTES IDENTIFICADOS:\n').bold = True
            limitantes = []
            if ndwi_mean < 0:
                limitantes.append('• Déficit hídrico severo - Prioridad: Riego inmediato')
            if ndre_mean < 0.25:
                limitantes.append('• Deficiencia nutricional (Nitrógeno) - Prioridad: Fertilización urgente')
            if msi_mean > 1.2:
                limitantes.append('• Estrés hídrico por alta temperatura - Prioridad: Manejo microclimático')
            if df_int.loc[3, 'Área (ha)'] > 0:
                limitantes.append('• Riesgo de erosión en zonas de pendiente - Prioridad: Prácticas conservacionistas')

            if limitantes:
                for lim in limitantes:
                    concl.add_run(f'{lim}\n')
            else:
                concl.add_run('• No se detectaron limitantes críticos\n')

            concl.add_run('\n')

            concl.add_run('RECOMENDACIÓN FINAL:\n').bold = True
            run_final = concl.add_run(
                'Implementar INMEDIATAMENTE las acciones propuestas priorizando las zonas rojas del mapa de intervención. '
            )
            run_final.bold = True

            concl.add_run('Establecer un programa de monitoreo satelital mensual para evaluar la respuesta a las ')
            concl.add_run('intervenciones y ajustar dinámicamente las estrategias. Desarrollar un plan de manejo ')
            concl.add_run('diferenciado por zonas considerando la variabilidad espacial identificada. ')
            concl.add_run('Complementar con monitoreo en campo para validación de datos satelitales.')

            doc.add_paragraph()

            # Próximos pasos
            doc.add_heading('PRÓXIMOS PASOS RECOMENDADOS:', level=2)

            pasos = doc.add_paragraph()
            pasos.add_run('1. Semana 1-2:\n').bold = True
            pasos.add_run('   • Implementar riego y fertilización en zonas prioritarias\n')
            pasos.add_run('   • Instalar puntos de monitoreo en campo\n')
            pasos.add_run('   • Iniciar registro fotográfico de zonas críticas\n\n')

            pasos.add_run('2. Semana 3-4:\n').bold = True
            pasos.add_run('   • Evaluar respuesta inicial mediante nuevo análisis satelital\n')
            pasos.add_run('   • Ajustar dosis de riego/fertilización según resultados\n')
            pasos.add_run('   • Expandir intervenciones a zonas de prioridad media\n\n')

            pasos.add_run('3. Mes 2:\n').bold = True
            pasos.add_run('   • Análisis satelital completo de seguimiento\n')
            pasos.add_run('   • Análisis de suelos en zonas problemáticas\n')
            pasos.add_run('   • Evaluación económica de intervenciones\n\n')

            pasos.add_run('4. Monitoreo continuo:\n').bold = True
            pasos.add_run('   • Análisis mensual durante toda la temporada\n')
            pasos.add_run('   • Ajustes dinámicos del plan de manejo\n')
            pasos.add_run('   • Documentación de resultados para temporadas futuras\n')

            doc.add_paragraph()
            doc.add_paragraph()

            # Firma y cierre
            firma = doc.add_paragraph()
            firma.add_run('_' * 60 + '\n')
            firma.add_run('INFORME GENERADO POR AgroIA\n').bold = True
            firma.add_run('Análisis Satelital Profesional con Google Earth Engine\n')
            firma.add_run('Tecnología: Sentinel-2 L2A + SRTM DEM\n')
            firma.add_run(f'Fecha de generación: {datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")}\n')
            firma.add_run('\n')
            firma.add_run('Para consultas técnicas o análisis adicionales:\n')
            firma.add_run('Contacte a su asesor agrícola o visite agroia.com\n')
            firma.alignment = WD_ALIGN_PARAGRAPH.CENTER

            # Guardar documento
            informe_path = f"{self.output_folder}/Informe_AgroIA_Completo_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.docx"
            doc.save(informe_path)

            print(f"✅ Informe Word completo generado: {informe_path}")
            return informe_path

        except Exception as e:
            logger.error(f"Error generando informe completo: {e}")
            print(f"⚠️ Error en informe completo, generando versión básica...")
            return self.generar_informe_basico(df_est, df_int, stats_topo, area_total)

    def generar_informe_basico(self, df_est, df_int, stats_topo, area_total):
        """Genera un informe básico como fallback"""
        try:
            doc = Document()

            title = doc.add_heading('INFORME AGROIA - ANÁLISIS DE CULTIVOS', 0)
            title.alignment = WD_ALIGN_PARAGRAPH.CENTER

            doc.add_paragraph(f"Fecha: {datetime.datetime.now().strftime('%d/%m/%Y %H:%M')}")
            doc.add_paragraph(f"Área analizada: {area_total:,.1f} hectáreas")
            doc.add_page_break()

            doc.add_heading('RESUMEN EJECUTIVO', level=1)
            p = doc.add_paragraph()
            p.add_run(f"NDVI promedio: {df_est.loc['NDVI', 'Media']:.3f}\n")
            p.add_run(f"NDWI promedio: {df_est.loc['NDWI', 'Media']:.3f}\n")
            p.add_run(f"Zonas prioridad alta: {df_int.loc[0, 'Área (ha)']:.1f} ha\n")

            doc.add_heading('ESTADÍSTICAS DE ÍNDICES', level=1)
            tabla = doc.add_table(rows=len(df_est)+1, cols=5)
            tabla.style = 'Light Grid Accent 1'

            encabezados = ['Índice', 'Media', 'Mínimo', 'Máximo', 'Desv. Est.']
            for i, enc in enumerate(encabezados):
                tabla.rows[0].cells[i].text = enc
                tabla.rows[0].cells[i].paragraphs[0].runs[0].font.bold = True

            for i, indice in enumerate(df_est.index, 1):
                tabla.rows[i].cells[0].text = indice
                tabla.rows[i].cells[1].text = f'{df_est.loc[indice, "Media"]:.3f}'
                tabla.rows[i].cells[2].text = f'{df_est.loc[indice, "Mínimo"]:.3f}'
                tabla.rows[i].cells[3].text = f'{df_est.loc[indice, "Máximo"]:.3f}'
                tabla.rows[i].cells[4].text = f'{df_est.loc[indice, "Desviación"]:.3f}'

            doc.add_heading('PLAN DE INTERVENCIÓN', level=1)
            tabla_int = doc.add_table(rows=len(df_int)+1, cols=4)
            tabla_int.style = 'Light Grid Accent 1'

            tabla_int.rows[0].cells[0].text = 'Prioridad'
            tabla_int.rows[0].cells[1].text = 'Área (ha)'
            tabla_int.rows[0].cells[2].text = '% Total'
            tabla_int.rows[0].cells[3].text = 'Acción Recomendada'

            for cell in tabla_int.rows[0].cells:
                cell.paragraphs[0].runs[0].font.bold = True

            for i, row in df_int.iterrows():
                tabla_int.rows[i+1].cells[0].text = row['Prioridad']
                tabla_int.rows[i+1].cells[1].text = f"{row['Área (ha)']:.1f}"
                tabla_int.rows[i+1].cells[2].text = f"{row['% del Total']:.1f}%"
                tabla_int.rows[i+1].cells[3].text = row['Acción Recomendada']

            informe_path = f"{self.output_folder}/informe_agroia_basico_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.docx"
            doc.save(informe_path)

            print(f"✅ Informe básico generado: {informe_path}")
            return informe_path

        except Exception as e:
            logger.error(f"❌ Error generando informe básico: {e}")
            return None

# %%
# CLASE BOT TELEGRAM COMPLETA
class AgroIABot:
    def __init__(self, token):
        self.token = token
        self.analizador = AnalizadorAgricolaCompleto()
        self.application = Application.builder().token(token).build()

        # Registrar manejadores
        self.application.add_handler(CommandHandler("start", self.start))
        self.application.add_handler(CommandHandler("help", self.help))
        self.application.add_handler(CommandHandler("analizar", self.analizar_coordenadas))
        self.application.add_handler(MessageHandler(filters.LOCATION, self.handle_location))
        self.application.add_handler(MessageHandler(filters.Document.ALL, self.handle_document))
        self.application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, self.handle_text))

    async def start(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Mensaje de bienvenida"""
        welcome_text = """
🤖 *Bienvenido a AgroIA - Análisis Profesional Completo*

*Tu Asistente Agrícola Inteligente con Sentinel-2*

📡 *¿Qué puedo hacer por ti?*
• Análisis completo con 8 índices espectrales (NDVI, NDWI, NDRE, EVI, SAVI, MSI, NDWI, NBR)
• Procesamiento robusto de archivos KML/KMZ
• Identificación precisa de zonas críticas
• Informe técnico profesional completo con interpretación experta
• Gráficos de alta calidad para presentaciones
• Recomendaciones técnicas detalladas y accionables

📍 *¿Cómo usar el bot?*
1️⃣ Envíame tu *ubicación GPS* 📍 (desde tu celular)
2️⃣ O envíame un archivo *KML/KMZ* 📁 con tu área de interés
3️⃣ O usa el comando: /analizar latitud,longitud 🔢

🌱 *Índices espectrales incluidos:*
• NDVI - Vigor y biomasa vegetal
• GNDVI - Contenido de clorofila
• NDRE - Estado nutricional (Nitrógeno)
• EVI - Vegetación densa
• SAVI - Corrección por suelo
• MSI - Estrés hídrico
• NDWI - Contenido de humedad
• NBR - Áreas degradadas

📊 *Tu informe incluirá:*
✅ Análisis exhaustivo de salud vegetal
✅ Plan de intervención priorizado con mapas
✅ 4 gráficos profesionales de alta resolución
✅ Interpretación experta de cada índice
✅ Recomendaciones técnicas detalladas
✅ Diagnóstico topográfico completo
✅ Acciones inmediatas paso a paso

⏱️ *Tiempo de procesamiento:* 3-5 minutos

*¡Envía tu ubicación o archivo KML ahora para comenzar!*

_Escribe /help para ver instrucciones detalladas_
        """
        await update.message.reply_text(welcome_text, parse_mode='Markdown')

    async def help(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Mostrar ayuda completa"""
        help_text = """
🆘 *Ayuda Completa de AgroIA*

📍 *OPCIÓN 1: Enviar ubicación GPS*
1. Toca el botón de clip 📎 (abajo a la izquierda)
2. Selecciona "Ubicación" 📍
3. Elige "Compartir mi ubicación actual"
4. ¡Listo! El bot analizará 5 km alrededor

🗂️ *OPCIÓN 2: Enviar archivo KML/KMZ*
1. Prepara tu archivo KML o KMZ (desde Google Earth, QGIS, etc.)
2. Envíalo como documento al bot 📁
3. El bot procesará automáticamente el área definida
4. *Formatos soportados:* .kml y .kmz

🔢 *OPCIÓN 3: Comando con coordenadas*
Formato: /analizar latitud,longitud
Ejemplo: /analizar -34.6037,-58.3816
(Analizará 5 km alrededor del punto)

📊 *¿Qué recibirás?*
• Informe Word completo (15-20 páginas)
• 4 gráficos profesionales en alta resolución
• Interpretación experta de cada índice
• Plan de intervención georeferenciado
• Recomendaciones técnicas accionables

⚠️ *Solución de problemas:*

*Si el KML falla:*
• Verifica que el archivo sea válido en Google Earth
• Asegúrate de que contenga polígonos o puntos
• Prueba exportarlo nuevamente
• Como alternativa, usa coordenadas GPS

*Si el análisis tarda:*
• Es normal, procesamos datos satelitales reales
• Tiempo estimado: 3-5 minutos
• No envíes múltiples solicitudes simultáneas

*Si no hay imágenes disponibles:*
• El área puede estar muy nublada
• Intenta con otra fecha o área cercana

⏱️ *Tiempo de procesamiento:* 3-5 minutos
🌍 *Cobertura:* Mundial (con imágenes Sentinel-2)
📅 *Período de análisis:* Enero - Junio 2025

💡 *Tip:* Para mejores resultados, usa archivos KML bien definidos o coordenadas precisas del centro de tu cultivo.

*¿Listo para comenzar? Envía tu ubicación o archivo KML ahora!*
        """
        await update.message.reply_text(help_text, parse_mode='Markdown')

    async def analizar_coordenadas(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Manejar comando /analizar con coordenadas"""
        if not context.args:
            await update.message.reply_text(
                "❌ *Formato incorrecto*\n\n"
                "*Uso correcto:*\n"
                "/analizar latitud,longitud\n\n"
                "*Ejemplos:*\n"
                "• /analizar -34.6037,-58.3816\n"
                "• /analizar 40.7128,-74.0060\n\n"
                "_Nota: Usa punto (.) como separador decimal_",
                parse_mode='Markdown'
            )
            return

        try:
            coords_text = " ".join(context.args)
            partes = coords_text.split(',')
            if len(partes) != 2:
                await update.message.reply_text(
                    "❌ Formato incorrecto\n"
                    "Usa: /analizar latitud,longitud\n"
                    "Ejemplo: /analizar -34.6037,-58.3816"
                )
                return

            lat = float(partes[0].strip())
            lon = float(partes[1].strip())

            if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
                await update.message.reply_text(
                    "❌ Coordenadas fuera de rango\n"
                    "Latitud debe estar entre -90 y 90\n"
                    "Longitud debe estar entre -180 y 180"
                )
                return

            await self.procesar_coordenadas(update, context, lat, lon)

        except ValueError:
            await update.message.reply_text(
                "❌ Coordenadas inválidas\n"
                "Asegúrate de usar números válidos\n"
                "Ejemplo: /analizar -34.6037,-58.3816"
            )
        except Exception as e:
            await update.message.reply_text(f"❌ Error: {str(e)}")

    async def handle_location(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Manejar ubicación GPS"""
        location = update.message.location
        lat = location.latitude
        lon = location.longitude
        await self.procesar_coordenadas(update, context, lat, lon)

    async def procesar_coordenadas(self, update: Update, context: ContextTypes.DEFAULT_TYPE, lat: float, lon: float):
        """Procesar coordenadas y ejecutar análisis completo"""
        processing_msg = await update.message.reply_text(
            f"📍 *Ubicación recibida*\n"
            f"Latitud: {lat:.6f}\n"
            f"Longitud: {lon:.6f}\n\n"
            f"🔄 *Iniciando análisis completo...*\n"
            f"⏳ Esto tomará 3-5 minutos\n\n"
            f"_Procesando imágenes Sentinel-2..._",
            parse_mode='Markdown'
        )

        try:
            geometria = self.analizador.crear_geometria_desde_coordenadas(lat, lon)
            area_ha = geometria.area().divide(10000).getInfo()

            await processing_msg.edit_text(
                f"📍 *Área identificada*\n"
                f"Ubicación: {lat:.6f}, {lon:.6f}\n"
                f"Extensión: {area_ha:.1f} hectáreas\n"
                f"Radio: 5 km\n\n"
                f"📡 *Descargando imágenes satelitales...*\n"
                f"⏳ Sentinel-2 L2A procesando..."
            )

            informe_path, error = self.analizador.analizar_area(geometria)

            if error:
                await processing_msg.edit_text(
                    f"❌ *Error en el análisis*\n\n"
                    f"{error}\n\n"
                    f"💡 *Sugerencias:*\n"
                    f"• Intenta con otra ubicación cercana\n"
                    f"• Verifica que haya cobertura Sentinel-2\n"
                    f"• El área puede estar muy nublada",
                    parse_mode='Markdown'
                )
                return

            await self.enviar_resultados_completos(update, context, informe_path, f"ubicación GPS ({lat:.4f}, {lon:.4f})")

        except Exception as e:
            logger.error(f"Error procesando ubicación: {e}")
            await processing_msg.edit_text(
                f"❌ *Error en el análisis*\n\n"
                f"Detalles: {str(e)}\n\n"
                f"Por favor intenta nuevamente o contacta soporte.",
                parse_mode='Markdown'
            )

    async def handle_document(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Manejar archivos KML/KMZ"""
        document = update.message.document
        file_name = document.file_name

        if not file_name.lower().endswith(('.kml', '.kmz')):
            await update.message.reply_text(
                "❌ *Formato no soportado*\n\n"
                "Solo acepto archivos:\n"
                "• *.kml* (Google Earth, QGIS)\n"
                "• *.kmz* (Google Earth comprimido)\n\n"
                "💡 *Cómo exportar desde Google Earth:*\n"
                "1. Selecciona tu polígono/área\n"
                "2. Clic derecho → Guardar lugar como\n"
                "3. Elige formato KML o KMZ\n"
                "4. Envíamelo aquí",
                parse_mode='Markdown'
            )
            return

        processing_msg = await update.message.reply_text(
            f"📁 *Archivo recibido:* {file_name}\n\n"
            f"🔄 *Procesando geometría KML...*\n"
            f"⏳ Esto puede tomar 1-2 minutos\n\n"
            f"_Extrayendo coordenadas..._",
            parse_mode='Markdown'
        )

        try:
            file = await context.bot.get_file(document.file_id)
            file_bytes = await file.download_as_bytearray()

            await processing_msg.edit_text(
                f"📁 *Procesando {file_name}...*\n\n"
                f"🔍 Analizando estructura del archivo\n"
                f"📐 Extrayendo polígonos y coordenadas\n"
                f"⏳ Un momento..."
            )

            geometria = self.analizador.procesar_kml(file_bytes, file_name)

            if not geometria:
                await processing_msg.edit_text(
                    "❌ *Error procesando KML*\n\n"
                    "⚠️ *El archivo no pudo ser procesado*\n\n"
                    "*Posibles causas:*\n"
                    "• Archivo KML corrupto o mal formado\n"
                    "• No contiene polígonos válidos\n"
                    "• Formato no estándar\n\n"
                    "*Soluciones:*\n"
                    "1. Abre el archivo en Google Earth y verifica que se vea bien\n"
                    "2. Exporta nuevamente como KML estándar\n"
                    "3. O envía tu ubicación GPS como alternativa\n\n"
                    "_Usa /help para más información_",
                    parse_mode='Markdown'
                )
                return

            area_ha = geometria.area().divide(10000).getInfo()
            await processing_msg.edit_text(
                f"✅ *KML procesado correctamente*\n\n"
                f"📁 Archivo: {file_name}\n"
                f"📐 Área: {area_ha:.1f} hectáreas\n\n"
                f"📡 *Ejecutando análisis satelital completo...*\n"
                f"⏳ 3-5 minutos restantes\n\n"
                f"_Descargando imágenes Sentinel-2..._"
            )

            informe_path, error = self.analizador.analizar_area(geometria)

            if error:
                await processing_msg.edit_text(
                    f"❌ *Error en el análisis*\n\n"
                    f"{error}\n\n"
                    f"El archivo KML se procesó bien, pero:\n"
                    f"• Puede no haber imágenes disponibles\n"
                    f"• El área puede estar muy nublada\n"
                    f"• Intenta con otra área o fecha",
                    parse_mode='Markdown'
                )
                return

            await self.enviar_resultados_completos(update, context, informe_path, f"archivo {file_name}")

        except Exception as e:
            logger.error(f"Error procesando documento: {e}")
            await processing_msg.edit_text(
                f"❌ *Error procesando archivo*\n\n"
                f"Detalles técnicos: {str(e)}\n\n"
                f"💡 *Sugerencias:*\n"
                f"• Verifica que el archivo sea válido\n"
                f"• Intenta exportarlo nuevamente\n"
                f"• O usa coordenadas GPS: /analizar lat,lon",
                parse_mode='Markdown'
            )

    async def handle_text(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Manejar mensajes de texto"""
        text = update.message.text.lower()

        # Detectar si menciona coordenadas
        if any(word in text for word in ['coord', 'lat', 'lon', 'gps', 'ubicación', 'ubicacion']):
            await update.message.reply_text(
                "📍 *Para enviar ubicación GPS:*\n\n"
                "*Desde el celular:*\n"
                "1. Toca el clip 📎 (abajo izquierda)\n"
                "2. Selecciona 'Ubicación' 📍\n"
                "3. Toca 'Compartir mi ubicación'\n\n"
                "*Con coordenadas:*\n"
                "/analizar -34.6037,-58.3816\n\n"
                "_Escribe /help para más opciones_",
                parse_mode='Markdown'
            )
            return

        # Respuesta general
        await update.message.reply_text(
            "🤖 *AgroIA - Tu Asistente Agrícola*\n\n"
            "*¿Cómo puedo ayudarte?*\n\n"
            "📍 Envía tu *ubicación GPS*\n"
            "📁 O un archivo *KML/KMZ*\n"
            "🔢 O usa: /analizar lat,lon\n\n"
            "*Recibirás:*\n"
            "📄 Informe técnico completo\n"
            "📊 8 índices espectrales\n"
            "🎯 Plan de intervención\n"
            "📈 Gráficos profesionales\n"
            "💡 Recomendaciones técnicas\n\n"
            "_Escribe /help para instrucciones completas_",
            parse_mode='Markdown'
        )

    async def enviar_resultados_completos(self, update: Update, context: ContextTypes.DEFAULT_TYPE, informe_path, fuente):
        """Envía todos los resultados del análisis"""
        try:
            if not informe_path or not os.path.exists(informe_path):
                await update.message.reply_text("❌ Error: No se pudo generar el informe")
                return

            success_msg = await update.message.reply_text(
                f"✅ *¡Análisis completado exitosamente!*\n\n"
                f"📊 Informe generado desde: {fuente}\n"
                f"📁 Enviando archivos...\n\n"
                f"_Por favor espera, los archivos son grandes_",
                parse_mode='Markdown'
            )

            # Enviar informe Word principal
            with open(informe_path, 'rb') as doc_file:
                await update.message.reply_document(
                    document=InputFile(doc_file, filename=f"Informe_AgroIA_{datetime.datetime.now().strftime('%Y%m%d_%H%M')}.docx"),
                    caption="📄 *INFORME TÉCNICO COMPLETO*\n\nAnálisis profesional con 8 índices espectrales, gráficos, mapas y recomendaciones detalladas.",
                    parse_mode='Markdown'
                )

            # Enviar gráficos
            imagenes_folder = f"{self.analizador.output_folder}/imagenes"
            if os.path.exists(imagenes_folder):
                graficos_enviados = 0
                for img_file in sorted(os.listdir(imagenes_folder)):
                    if img_file.endswith('.png'):
                        img_path = os.path.join(imagenes_folder, img_file)
                        with open(img_path, 'rb') as img:
                            if 'indices' in img_file:
                                caption = "📊 *GRÁFICO 1: Comparación de Índices*\n\nValores promedio de los 8 índices espectrales calculados"
                            elif 'intervencion' in img_file:
                                caption = "🎯 *GRÁFICO 2: Plan de Intervención*\n\nDistribución de áreas por prioridad y porcentajes"
                            elif 'salud' in img_file:
                                caption = "🌱 *GRÁFICO 3: Salud de Cultivos*\n\nAnálisis detallado de NDVI, NDWI, NDRE y MSI"
                            elif 'topografia' in img_file:
                                caption = "🗻 *GRÁFICO 4: Topografía y Riesgos*\n\nElevación, pendientes y zonas de erosión"
                            else:
                                caption = "📈 Gráfico de análisis"

                            await update.message.reply_photo(
                                photo=InputFile(img),
                                caption=caption,
                                parse_mode='Markdown'
                            )
                            graficos_enviados += 1

            # Mensaje final con resumen
            await success_msg.edit_text(
                f"🎉 *¡ANÁLISIS COMPLETADO CON ÉXITO!*\n\n"
                f"✅ *Archivos generados:*\n"
                f"• 📄 Informe Word completo (15-20 páginas)\n"
                f"• 📊 {graficos_enviados} gráficos profesionales\n"
                f"• 🎯 Plan de intervención georeferenciado\n"
                f"• 💡 Recomendaciones técnicas accionables\n\n"
                f"📍 *¿Necesitas analizar otra área?*\n"
                f"Envía nueva ubicación GPS o archivo KML\n\n"
                f"📚 *¿Tienes preguntas?*\n"
                f"Escribe /help para más información\n\n"
                f"🌱 *AgroIA* - Análisis profesional al alcance de tu mano\n"
                f"_Powered by Sentinel-2 & Google Earth Engine_",
                parse_mode='Markdown'
            )

        except Exception as e:
            logger.error(f"Error enviando resultados: {e}")
            await update.message.reply_text(
                f"❌ *Error enviando archivos*\n\n"
                f"Los archivos se generaron pero hubo un problema al enviarlos.\n\n"
                f"Detalles: {str(e)}",
                parse_mode='Markdown'
            )

# %%
# CONFIGURACIÓN Y EJECUCIÓN
def setup_ngrok(port=8443):
    """Configura ngrok"""
    try:
        ngrok.kill()
        public_url = ngrok.connect(port, bind_tls=True).public_url
        print(f"✅ Ngrok configurado: {public_url}")
        return public_url
    except Exception as e:
        print(f"⚠️ Error configurando ngrok: {e}")
        return None

async def main_async():
    """Función principal asíncrona"""
    print("🔧 Configurando ngrok...")
    public_url = setup_ngrok(PORT)

    if public_url:
        print(f"🌐 URL pública: {public_url}")

    print("🤖 Iniciando AgroIA Bot COMPLETO...")
    bot = AgroIABot(TELEGRAM_TOKEN)

    # Configurar comandos
    commands = [
        BotCommand("start", "Iniciar AgroIA Bot"),
        BotCommand("help", "Ayuda completa"),
        BotCommand("analizar", "Analizar por coordenadas: /analizar lat,lon")
    ]

    await bot.application.bot.set_my_commands(commands)

    print("🚀 Bot funcionando con polling...")
    print("✅ Procesamiento KML funcional")
    print("📊 Informes completos activados")
    print("🔗 Ve a Telegram y escribe /start")
    print("⏹️  Presiona Ctrl+C para detener")

    try:
        await bot.application.initialize()
        await bot.application.start()
        await bot.application.updater.start_polling()

        # Mantener el bot corriendo
        while True:
            await asyncio.sleep(3600)

    except KeyboardInterrupt:
        print("\n🛑 Deteniendo bot...")
    except Exception as e:
        print(f"❌ Error en el bot: {e}")
    finally:
        if bot.application.updater:
            await bot.application.updater.stop()
        await bot.application.stop()
        await bot.application.shutdown()

def main():
    """Función principal para ejecutar"""
    print("="*60)
    print("🚀 INICIANDO AGROIA BOT COMPLETO")
    print("="*60)
    print("📊 Análisis completo con 8 índices espectrales")
    print("📄 Informe Word profesional de 15-20 páginas")
    print("🗺️  Procesamiento robusto de KML/KMZ")
    print("📈 Gráficos de alta calidad")
    print("💡 Recomendaciones técnicas detalladas")
    print("="*60)

    try:
        # Autenticar Earth Engine
        try:
            ee.Initialize(project='eddycc66')
            print("✅ Earth Engine inicializado")
        except:
            print("⚠️ Autenticando Earth Engine...")
            ee.Authenticate()
            ee.Initialize(project='eddycc66')
            print("✅ Earth Engine autenticado e inicializado")

        # Ejecutar bot
        loop = asyncio.get_event_loop()
        if loop.is_running():
            loop.create_task(main_async())
        else:
            loop.run_until_complete(main_async())
    except Exception as e:
        print(f"❌ Error ejecutando el bot: {e}")
        import traceback
        traceback.print_exc()

# %%
# EJECUCIÓN FINAL
if __name__ == "__main__":
    print("\n" + "="*60)
    print("🌱 AGROIA BOT - ANÁLISIS AGRÍCOLA PROFESIONAL")
    print("="*60)
    print("\n📱 Abre Telegram y busca tu bot")
    print("📍 Envía una ubicación GPS o archivo KML")
    print("⏱️  El análisis toma 3-5 minutos")
    print("\n🎯 Recibirás:")
    print("   • Informe Word completo (15-20 páginas)")
    print("   • 4 gráficos profesionales")
    print("   • Plan de intervención detallado")
    print("   • Recomendaciones técnicas")
    print("\n" + "="*60)
    print("\n🚀 INICIANDO BOT AHORA...\n")

    main()